Здесь создается submission для тестовых данных

In [1]:
import pandas as pd
from transformers import AutoModelForTokenClassification, AutoTokenizer
import torch
from tqdm import tqdm

Подгружаем и разархивируем дообученную модель

In [9]:
!unzip -o model.zip -d ./trained_model

Archive:  model.zip
   creating: ./trained_model/prosto_dataset_base/
  inflating: ./trained_model/prosto_dataset_base/model.safetensors  
  inflating: ./trained_model/prosto_dataset_base/training_args.bin  
  inflating: ./trained_model/prosto_dataset_base/vocab.txt  
  inflating: ./trained_model/prosto_dataset_base/config.json  
  inflating: ./trained_model/prosto_dataset_base/tokenizer_config.json  
  inflating: ./trained_model/prosto_dataset_base/tokenizer.json  
  inflating: ./trained_model/prosto_dataset_base/special_tokens_map.json  


In [11]:
model = AutoModelForTokenClassification.from_pretrained("./trained_model/prosto_dataset_base")
tokenizer = AutoTokenizer.from_pretrained("./trained_model/prosto_dataset_base")

In [12]:
def load_txt_to_df(file_path: str, sep: str = ",", columns=("id", "text")) -> pd.DataFrame:
    """
    Загружает txt-файл в DataFrame.
    Ожидается формат: id,text.

    Принимает:
        file_path (str): путь к txt-файлу
        sep (str): разделитель (по умолчанию ",")
        columns: имена колонок (по умолчанию ("id", "text"))

    Возвращает:
        pd.DataFrame
    """
    with open(file_path, "r", encoding="utf-8") as f:
        lines = [line.strip() for line in f]

    data = [line.split(sep, 1) for line in lines]
    df = pd.DataFrame(data, columns=columns)
    df = df.iloc[1:].reset_index(drop=True)
    return df


In [13]:
def restore_spaces(text, model, tokenizer, device="cuda"):
    """
    Восстанавливает пробелы в тексте по предсказаниям модели.

    Принимает:
        text (str): строка без пробелов
        model: обученная модель
        tokenizer: токенизатор
        device (str): устройство для вычислений ("cuda" или "cpu")

    Возвращает:
        str: восстановленный текст с пробелами
    """

    model.to(device)
    model.eval()

    tokens = list(text)
    enc = tokenizer(tokens, is_split_into_words=True, return_tensors="pt", truncation=True, max_length=128)
    enc = {k: v.to(device) for k, v in enc.items()}

    with torch.no_grad():
        outputs = model(**enc)
        preds = outputs.logits.argmax(-1).squeeze().tolist()

    result = ""
    for ch, p in zip(tokens, preds[:len(tokens)]):
        result += ch
        if p == 1:
            result += " "
    return result


Пример восстановления пробелов для запроса "Сдам квартиру"

In [14]:
example = restore_spaces('сдамквартиру', model, tokenizer)
print(example)

сдам квартиру


Загружаем тестовый даатсет

In [35]:
test_df_path = '/content/dataset_1937770_3.txt'
test_df = load_txt_to_df(test_df_path)

In [36]:
test_df.head(2)

,id,text
0,0,куплюайфон14про
1,1,ищудомвПодмосковье


In [37]:
# Убедимся, что модель на нужном устройстве
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Создаем новый столбец с предсказанным текстом
test_df['text_spaces'] = [
    restore_spaces(text, model, tokenizer, device=device)
    for text in tqdm(test_df['text'], desc="Restoring spaces")
]


Restoring spaces: 100%|██████████| 1005/1005 [00:10<00:00, 99.23it/s] 


In [38]:
test_df.head()

,id,text,text_spaces
0,0,куплюайфон14про,куплю айфон 14 про
1,1,ищудомвПодмосковье,ищу дом в Подмосковье
2,2,сдаюквартирусмебельюитехникой,сдаю квартиру с мебелью и техникой
3,3,новыйдивандоставканедорого,новый диван доставка недорого
4,4,отдамдаромкошку,отдам даром кошку


In [39]:
def get_space_indices(text):
    """
    Возвращает индексы всех пробелов в строке

    Принимает:
        text (str): строка с пробелами

    Возвращает:
        list[int]: список индексов (позиции символов в строке),
    """
    return [i for i, ch in enumerate(text) if ch == " "]


# Создаем новый столбец с индексами пробелов
test_df['space_pos'] = test_df['text_spaces'].apply(get_space_indices)

In [40]:
test_df.head()

,id,text,text_spaces,space_pos
0,0,куплюайфон14про,куплю айфон 14 про,"[5, 11, 14]"
1,1,ищудомвПодмосковье,ищу дом в Подмосковье,"[3, 7, 9]"
2,2,сдаюквартирусмебельюитехникой,сдаю квартиру с мебелью и техникой,"[4, 13, 15, 23, 25]"
3,3,новыйдивандоставканедорого,новый диван доставка недорого,"[5, 11, 20]"
4,4,отдамдаромкошку,отдам даром кошку,"[5, 11]"


In [41]:
test_df['text_spaces'].tolist()

['куплю айфон 14 про',
 'ищу дом в Подмосковье',
 'сдаю квартиру с мебелью и техникой',
 'новый диван доставка недорого',
 'отдам даром кошку',
 'работа в Москве удаленно',
 'куплю телевизор Philips',
 'ищу грузчиков для переезда',
 'ремонт квартир под ключ',
 'куплю ноутбук HP',
 'ищу квартиру у метро',
 'новая микроволновка Samsung',
 'срочно продам велосипед',
 'куплю гитару Fender',
 'ищу репетитора по биологии',
 'сдаю гараж на длительный срок',
 'куплю диван бу',
 'ищу мастера по ремонту холодильников',
 'новый шкаф доставка сегодня',
 'куплю Xbox One',
 'ищу подработку по вечерам',
 'сдам комнату студентке',
 'куплю старую книгу',
 'ищу собаку лабрадор',
 'новый телефон Xiaomi 13',
 'куплю Playstation 5 диск',
 'ищу комнату в центре города',
 'срочно нужна няня ребенку',
 'куплю стиральную машину Indesit',
 'ищу детскую кроватку бу',
 'новая куртка доставка',
 'куплю велосипед Merida',
 'ищу врача офтальмолога',
 'сдаю квартиру в центре Москвы',
 'куплю холодильник Samsung',
 'и

In [45]:
def get_space_positions(no_space_text, spaced_text):
    """
    Возвращает индексы символов в строке без пробелов,
    после которых должны стоять пробелы в строке с пробелами.

    Принимает:
        no_space_text (str): строка без пробелов
        spaced_text (str): строка с пробелами

    Возвращает:
        list[int]: список индексов символов (по строке без пробелов),
                   после которых в исходной строке встречался пробел.

    Пример:
        >>> get_space_positions("helloworld", "hello world")
        [4]
        # потому что пробел стоит после символа "t" (индекс 4)
    """
    positions = []
    j = 0
    for char in spaced_text:
        if char == " ":
            positions.append(j - 1)  # пробел после предыдущего символа
        else:
            j += 1
    return positions


test_df["predicted_positions"] = test_df.apply(
    lambda row: get_space_positions(row["text"], row["text_spaces"]), axis=1
)

Добавим +1 для корректности принятия решения

In [48]:
test_df["predicted_positions"] = test_df["predicted_positions"].apply(
    lambda lst: [x + 1 for x in lst] if isinstance(lst, list) else lst
)

In [49]:
test_df.head()

,id,text,text_spaces,space_pos,predicted_positions
0,0,куплюайфон14про,куплю айфон 14 про,"[5, 11, 14]","[5, 10, 12]"
1,1,ищудомвПодмосковье,ищу дом в Подмосковье,"[3, 7, 9]","[3, 6, 7]"
2,2,сдаюквартирусмебельюитехникой,сдаю квартиру с мебелью и техникой,"[4, 13, 15, 23, 25]","[4, 12, 13, 20, 21]"
3,3,новыйдивандоставканедорого,новый диван доставка недорого,"[5, 11, 20]","[5, 10, 18]"
4,4,отдамдаромкошку,отдам даром кошку,"[5, 11]","[5, 10]"


In [50]:
test_df["predicted_positions"] = test_df["predicted_positions"].apply(lambda x: str(x) if isinstance(x, list) else x)


In [51]:
type(test_df['predicted_positions'][1])

str

In [52]:
test_df.head()

,id,text,text_spaces,space_pos,predicted_positions
0,0,куплюайфон14про,куплю айфон 14 про,"[5, 11, 14]","[5, 10, 12]"
1,1,ищудомвПодмосковье,ищу дом в Подмосковье,"[3, 7, 9]","[3, 6, 7]"
2,2,сдаюквартирусмебельюитехникой,сдаю квартиру с мебелью и техникой,"[4, 13, 15, 23, 25]","[4, 12, 13, 20, 21]"
3,3,новыйдивандоставканедорого,новый диван доставка недорого,"[5, 11, 20]","[5, 10, 18]"
4,4,отдамдаромкошку,отдам даром кошку,"[5, 11]","[5, 10]"


Создаем submission

In [ ]:
test_df[["id", "predicted_positions"]].to_csv("submission.csv", index=False, encoding="utf-8")

Функция для првоерок

In [86]:
test1 = test_df[['text']].copy()
test1.head()

,text
0,куплюайфон14про
1,ищудомвПодмосковье
2,сдаюквартирусмебельюитехникой
3,новыйдивандоставканедорого
4,отдамдаромкошку


In [87]:
def your_tests(text_series: pd.Series):
  """
  Принимает:
      text_series (pd.Series): столбец с текстами без пробелов
      model: обученная модель
      tokenizer: токенизатор

  Возвращает:
      pd.DataFrame с колонками:
        - text: исходный текст без пробелов
        - text_spaces: текст с восстановленными пробелами
        - predicted_positions: индексы символов (+1), после которых предсказан пробел
        - space_pos: индексы реальных пробелов в восстановленном тексте
  """
  df = pd.DataFrame({"text": text_series.copy()})

  df['text_spaces'] = [
      restore_spaces(text, model, tokenizer, device=device)
      for text in tqdm(df['text'], desc="Restoring spaces")]

  df["predicted_positions"] = df.apply(
      lambda row: get_space_positions(row["text"], row["text_spaces"]), axis=1)

  df["predicted_positions"] = df["predicted_positions"].apply(
      lambda lst: [x + 1 for x in lst] if isinstance(lst, list) else lst)

  df['space_pos'] = df['text_spaces'].apply(get_space_indices)

  return df



In [88]:
test1 = your_tests(test1["text"])

Restoring spaces: 100%|██████████| 1005/1005 [00:15<00:00, 66.51it/s] 


In [83]:
test1.head()

,text,text_spaces,predicted_positions,space_pos
0,куплюайфон14про,куплю айфон 14 про,"[5, 10, 12]","[5, 11, 14]"
1,ищудомвПодмосковье,ищу дом в Подмосковье,"[3, 6, 7]","[3, 7, 9]"
2,сдаюквартирусмебельюитехникой,сдаю квартиру с мебелью и техникой,"[4, 12, 13, 20, 21]","[4, 13, 15, 23, 25]"
3,новыйдивандоставканедорого,новый диван доставка недорого,"[5, 10, 18]","[5, 11, 20]"
4,отдамдаромкошку,отдам даром кошку,"[5, 10]","[5, 11]"
